<a href="https://colab.research.google.com/github/Manos-Mak120/Makrakis-Manos/blob/main/Biden_df_articles_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import time
import random

from tqdm import tqdm

In [ ]:
def get_webpage_soup(url):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    return None
  else:
    return BeautifulSoup(response.text, 'html.parser')

In [ ]:
def get_webpage_soup(url):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    return None
  else:
    return BeautifulSoup(response.text, 'html.parser')
def get_article_elements_from_soup(soup, article_url):
    # title
    try:
        title = soup.find("h1").text.strip()
    except:
        title = ""

    # datetime
    try:
        datetime_str = soup.find("time")["datetime"]
        date_str = datetime_str.split("T")[0]
        time_str = datetime_str.split("T")[1]
    except:
        date_str = ""
        time_str = ""
    # section
    try:
        section_div = soup.find("div", class_="main-category-block") or soup.find("div", class_="main-category")
        section = section_div.text.strip() if section_div else ""
    except:
        section = ""

    # author
    try:
      author_tag = soup.find("a", class_="author-name") or soup.find("span", class_="author-name")
      author = author_tag.text.strip() if author_tag else ""
    except:
        author = ""
    # excerpt
    try:
        excerpt_div = soup.find("div", class_="p.main-intro") or soup.find(itemprop="description") or soup.find("div", class_="nx-excerpt")
        excerpt = excerpt_div.text.strip() if excerpt_div else ""
    except:
        excerpt = ""

    # text body
    try:
      content_div = soup.find("div", class_="main-text story-fulltext")
      elements = content_div.find_all(["p", "h2", "h3", "ul", "ol", "li"]) if content_div else []
      text_body = " ".join(el.get_text(strip=True) for el in elements if el.get_text(strip=True))
    except:
        text_body = ""

    # feature image
    try:
        img_block = soup.find("div", class_="article-image") or soup.find("figure")
        feat_img_link = img_block.find("img")["src"] if img_block and img_block.find("img") else ""
    except:
        feat_img_link = ""

    # image caption
    try:
        caption_div = soup.find("div", class_="image-caption") or soup.find("figcaption")
        image_caption = caption_div.text.strip() if caption_div else ""
    except:
        image_caption = ""

    articleD = {
        "article_url": article_url,
        "title": title,
        "date": date_str,
        "time": time_str,
        "section": section,
        "author": author,
        "excerpt": excerpt,
        "text_body": text_body,
        "feat_img_link": feat_img_link,
        "feat_img_cation": image_caption
    }

    return articleD

In [ ]:
def scrape_newsbomb_articles_pages(teasers_df):
  all_articles_dataL = []
  base_url = "https://www.newsbomb.gr"
  for idx, row in tqdm(teasers_df.iterrows(), total=teasers_df.shape[0], desc="Processing rows"):
    article_url_relative = row['article_url']
    article_url = base_url + article_url_relative
    soup = get_webpage_soup(article_url)
    if soup:
     articleD = get_article_elements_from_soup(soup, article_url)
     all_articles_dataL.append(articleD)
    delay = random.uniform(1, 3)
    time.sleep(delay)
  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetshed {len(df)} articles.")
  return df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
biden_teasers_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/biden_2020_df.csv")
biden_teasers_df.head()

,title,article_url,date,time,section,excerpt
0,ΗΠΑ: Ο Μπάιντεν δεσμεύεται να συγκροτήσει επιτ...,/kosmos/story/1087367/hpa-o-mpainten-desmeyeta...,01.06.2020,21:17,ΚΟΣΜΟΣ,"Ο Τζο Μπάιντεν, ο υποψήφιος των Δημοκρατικών γ..."
1,Εκλογές ΗΠΑ: Έρχονται μέτρα στα επόμενα ντιμπέ...,/kosmos/story/1122674/ekloges-hpa-erxontai-met...,01.10.2020,01:53,ΚΟΣΜΟΣ,"Η επιτροπή, που είναι επιφορτισμένη με την οργ..."
2,ΗΠΑ: Αρνητικός ο Τζο Μπάιντεν στο τεστ κορονοϊού,/kosmos/story/1123295/hpa-arnitikos-o-tzo-mpai...,02.10.2020,20:05,ΚΟΣΜΟΣ,Κορονοϊός - Όλες οι ειδήσεις: Ο Τζο Μπάιντεν κ...
3,ΗΠΑ: Ο Τζο Μπάιντεν εύχεται «ταχεία ανάρρωση» ...,/kosmos/story/1123251/o-tzo-mpainten-eyxetai-t...,02.10.2020,16:48,ΚΟΣΜΟΣ,Κορονοϊός - Όλες οι ειδήσεις: Ο υποψήφιος των ...
4,ΗΠΑ: «Ο Θεός να σε ευλογεί» - Το μήνυμα του Μπ...,/kosmos/story/1123371/hpa-o-theos-na-se-eyloge...,03.10.2020,03:20,ΚΟΣΜΟΣ,Ο Τζο Μπάιντεν έστειλε το δικό του μήνυμα στον...


In [ ]:
biden_teasers_df_test = biden_teasers_df.head(10)
biden_teasers_df_test

,title,article_url,date,time,section,excerpt
0,ΗΠΑ: Ο Μπάιντεν δεσμεύεται να συγκροτήσει επιτ...,/kosmos/story/1087367/hpa-o-mpainten-desmeyeta...,01.06.2020,21:17,ΚΟΣΜΟΣ,"Ο Τζο Μπάιντεν, ο υποψήφιος των Δημοκρατικών γ..."
1,Εκλογές ΗΠΑ: Έρχονται μέτρα στα επόμενα ντιμπέ...,/kosmos/story/1122674/ekloges-hpa-erxontai-met...,01.10.2020,01:53,ΚΟΣΜΟΣ,"Η επιτροπή, που είναι επιφορτισμένη με την οργ..."
2,ΗΠΑ: Αρνητικός ο Τζο Μπάιντεν στο τεστ κορονοϊού,/kosmos/story/1123295/hpa-arnitikos-o-tzo-mpai...,02.10.2020,20:05,ΚΟΣΜΟΣ,Κορονοϊός - Όλες οι ειδήσεις: Ο Τζο Μπάιντεν κ...
3,ΗΠΑ: Ο Τζο Μπάιντεν εύχεται «ταχεία ανάρρωση» ...,/kosmos/story/1123251/o-tzo-mpainten-eyxetai-t...,02.10.2020,16:48,ΚΟΣΜΟΣ,Κορονοϊός - Όλες οι ειδήσεις: Ο υποψήφιος των ...
4,ΗΠΑ: «Ο Θεός να σε ευλογεί» - Το μήνυμα του Μπ...,/kosmos/story/1123371/hpa-o-theos-na-se-eyloge...,03.10.2020,03:20,ΚΟΣΜΟΣ,Ο Τζο Μπάιντεν έστειλε το δικό του μήνυμα στον...
5,Εκλογές ΗΠΑ 2020 - Δημοσκόπηση: Προβάδισμα 7 μ...,/kosmos/story/1132775/ekloges-hpa-2020-dimosko...,01.11.2020,21:43,ΚΟΣΜΟΣ,Εκλογές ΗΠΑ 2020: Στην τελική ευθεία η μάχη Τρ...
6,Εκλογές ΗΠΑ 2020: Τραμπ εναντίον Μπάιντεν – Τι...,/kosmos/story/1132686/ekloges-hpa-2020-tramp-e...,01.11.2020,14:25,ΚΟΣΜΟΣ,Προεδρικές Εκλογές ΗΠΑ 2020: Ένα 48ωρο πριν απ...
7,Εκλογές ΗΠΑ 2020: Ο Τραμπ επιτίθεται στον «κοι...,/kosmos/story/1132681/ekloges-hpa-2020-o-tramp...,01.11.2020,13:55,ΚΟΣΜΟΣ,Εκλογές ΗΠΑ 2020: Ο Ντόναλντ Τραμπ υποστήριξε ...
8,Εκλογές ΗΠΑ 2020: Κορυφώνεται η μάχη Τραμπ vs ...,/kosmos/story/1132645/ekloges-hpa-2020-koryfon...,01.11.2020,11:16,ΚΟΣΜΟΣ,Εκλογές ΗΠΑ 2020: Δυο ημέρες πριν κλείσουν οι ...
9,Εκλογές ΗΠΑ 2020: Τραμπ vs Μπάιντεν - Ποιος θα...,/kosmos/story/1132276/ekloges-hpa-2020-tramp-v...,01.11.2020,08:27,ΚΟΣΜΟΣ,Η αντίστροφη μέτρηση για τις αμερικανικές προε...


In [ ]:
articles_df = scrape_newsbomb_articles_pages(biden_teasers_df)
articles_df

Processing rows: 100%|██████████| 46/46 [02:27<00:00,  3.21s/it]


Scraping completed. Fetshed 46 articles.


,article_url,title,date,time,section,author,excerpt,text_body,feat_img_link,feat_img_cation
0,https://www.newsbomb.gr/kosmos/story/1087367/h...,ΗΠΑ: Ο Μπάιντεν δεσμεύεται να συγκροτήσει επιτ...,2020-06-01,21:17:41+03:00,ΚΟΣΜΟΣ,Newsbomb,"Ο Τζο Μπάιντεν, ο υποψήφιος των Δημοκρατικών γ...",«Το τσιρότο το τράβηξαν απότομα αυτή η πανδημί...,https://img.bbmd.gr/img/1260/max/82/2020/06/01...,
1,https://www.newsbomb.gr/kosmos/story/1122674/e...,Εκλογές ΗΠΑ: Έρχονται μέτρα στα επόμενα ντιμπέ...,2020-10-01,01:53:25+03:00,ΚΟΣΜΟΣ,Newsbomb,"Η επιτροπή, που είναι επιφορτισμένη με την οργ...",«Το πρώτο ντιμπέιτ κατέστησε σαφές το γεγονός ...,https://img.bbmd.gr/img/1260/max/82/2020/09/30...,AP Photo
2,https://www.newsbomb.gr/kosmos/story/1123295/h...,ΗΠΑ: Αρνητικός ο Τζο Μπάιντεν στο τεστ κορονοϊού,2020-10-02,20:05:24+03:00,ΚΟΣΜΟΣ,Newsbomb,Κορονοϊός - Όλες οι ειδήσεις: Ο Τζο Μπάιντεν κ...,Αρνητική στην Covid-19 βγήκε και η υποψήφια αν...,https://img.bbmd.gr/img/1260/max/82/2020/10/02...,
3,https://www.newsbomb.gr/kosmos/story/1123251/o...,ΗΠΑ: Ο Τζο Μπάιντεν εύχεται «ταχεία ανάρρωση» ...,2020-10-02,16:48:48+03:00,ΚΟΣΜΟΣ,Newsbomb,Κορονοϊός - Όλες οι ειδήσεις: Ο υποψήφιος των ...,Η Τζιλ Μπάιντεν (η σύζυγός του) και εγώ στέλνο...,https://img.bbmd.gr/img/1260/max/82/2020/10/02...,
4,https://www.newsbomb.gr/kosmos/story/1123371/h...,ΗΠΑ: «Ο Θεός να σε ευλογεί» - Το μήνυμα του Μπ...,2020-10-03,03:20:10+03:00,ΚΟΣΜΟΣ,Newsbomb,Ο Τζο Μπάιντεν έστειλε το δικό του μήνυμα στον...,«Αυτές είναι οι Ηνωμένες Πολιτείες της Αμερική...,https://img.bbmd.gr/img/1260/max/82/2020/10/03...,
5,https://www.newsbomb.gr/kosmos/story/1132775/e...,Εκλογές ΗΠΑ 2020 - Δημοσκόπηση: Προβάδισμα 7 μ...,2020-11-01,21:43:04+02:00,ΚΟΣΜΟΣ,Newsbomb,Εκλογές ΗΠΑ 2020: Στην τελική ευθεία η μάχη Τρ...,Με τις αμερικανικές προεδρικές εκλογές να απέχ...,https://img.bbmd.gr/img/1260/max/82/2020/11/01...,AP
6,https://www.newsbomb.gr/kosmos/story/1132686/e...,Εκλογές ΗΠΑ 2020: Τραμπ εναντίον Μπάιντεν – Τι...,2020-11-01,14:25:49+02:00,ΚΟΣΜΟΣ,Χρήστος Γιαννούλης,Προεδρικές Εκλογές ΗΠΑ 2020: Ένα 48ωρο πριν απ...,Οι ψηφοφόροι στις ΗΠΑ θα αποφασίσουν στις 3 Νο...,https://img.bbmd.gr/img/1260/max/82/2020/11/01...,Τι δείχνουν οι δημοσκοπήσεις για τις εκλογές τ...
7,https://www.newsbomb.gr/kosmos/story/1132681/e...,Εκλογές ΗΠΑ 2020: Ο Τραμπ επιτίθεται στον «κοι...,2020-11-01,13:55:24+02:00,ΚΟΣΜΟΣ,Χρήστος Γιαννούλης,Εκλογές ΗΠΑ 2020: Ο Ντόναλντ Τραμπ υποστήριξε ...,Η μάχη τωνΠροεδρικών Εκλογών στις ΗΠΑκορυφώνετ...,https://img.bbmd.gr/img/1260/max/82/2020/11/01...,"Ο Αμερικανός πρόεδρος, Ντόναλντ Τραμπ\nAP Images"
8,https://www.newsbomb.gr/kosmos/story/1132645/e...,Εκλογές ΗΠΑ 2020: Κορυφώνεται η μάχη Τραμπ vs ...,2020-11-01,11:16:56+02:00,ΚΟΣΜΟΣ,Χρήστος Γιαννούλης,Εκλογές ΗΠΑ 2020: Δυο ημέρες πριν κλείσουν οι ...,«Φωτιά» έχει πάρει το επιτελείο τουΝτόναλντ Τρ...,https://img.bbmd.gr/img/1260/max/82/2020/11/01...,Λιγότερες από 48 ώρες για την τελική αναμέτρησ...
9,https://www.newsbomb.gr/kosmos/story/1132276/e...,Εκλογές ΗΠΑ 2020: Τραμπ vs Μπάιντεν - Ποιος θα...,2020-11-01,08:27:57+02:00,ΚΟΣΜΟΣ,Ελένη Κολιοφούτη,Η αντίστροφη μέτρηση για τις αμερικανικές προε...,Αντίστροφα μετράει η κλεψύδρα για την πολυαναμ...,https://img.bbmd.gr/img/1260/max/82/2020/10/30...,Στην τελική ευθεία η μάχη Τραμπ - Μπάιντεν για...


In [ ]:
articles_df['text_body'][0]  #έλεγχος ότι πήρε όλο το κείμενο

'«Το τσιρότο το τράβηξαν απότομα αυτή η πανδημία και αυτός ο πρόεδρος», είπε ο Μπάιντεν, αναφερόμενος στον κορονοϊό που πλήττει δυσανάλογα τους Αφροαμερικανούς και ορισμένες περιοχές των ΗΠΑ αλλά και στις δηλώσεις του Ρεπουμπλικανού μεγιστάνα για τις μειονότητες. «Πλέον κανείς δεν μπορεί να υποκρίνεται ότι δεν ξέρει τι συμβαίνει», πρόσθεσε ο πρώην αντιπρόεδρος του Μπαράκ Ομπάμα, ο οποίος επιχείρησε να εμφανιστεί ενωτικός σε αυτήν την συνάντηση που οργανώθηκε σε μια εκκλησία στην πόλη Ουίλμινγκτον, στο Ντέλαγουερ. «Το μίσος απλώς κρύβεται. Δεν εξαφανίζεται. Και όταν κάποιος που βρίσκεται στην εξουσία αναζωπυρώνει το κρυμμένο μίσος, εκείνο εμφανίζεται. Οι λέξεις ενός προέδρου είναι σημαντικές. Ενθαρρύνουν τους ανθρώπους να βγάζουν το βιτριόλι», δήλωσε. Ο 77χρονος Μπάιντεν αναφερόταν κυρίως στις δηλώσεις που είχε κάνει ο Τραμπ μετά τις ταραχές στο Σάρλοτσβιλ, τον Αύγουστο του 2017, μεταξύ των υπέρμαχων της λευκής φυλής και των αντιρατσιστών διαδηλωτών, όταν είπε ότι υπήρχαν «πολύ καλοί άν

In [ ]:
articles_df['feat_img_link'][0]

'https://img.bbmd.gr/img/1260/max/82/2020/06/01/mpaintn.jpg?t=wUkE4CMh0N7eX2rSzBsI4Q'

In [ ]:
articles_df = scrape_newsbomb_articles_pages(biden_teasers_df)
articles_df

Processing rows: 100%|██████████| 46/46 [02:25<00:00,  3.17s/it]


Scraping completed. Fetshed 46 articles.


,article_url,title,date,time,section,author,excerpt,text_body,feat_img_link,feat_img_cation
0,https://www.newsbomb.gr/kosmos/story/1087367/h...,ΗΠΑ: Ο Μπάιντεν δεσμεύεται να συγκροτήσει επιτ...,2020-06-01,21:17:41+03:00,ΚΟΣΜΟΣ,Newsbomb,"Ο Τζο Μπάιντεν, ο υποψήφιος των Δημοκρατικών γ...",«Το τσιρότο το τράβηξαν απότομα αυτή η πανδημί...,https://img.bbmd.gr/img/1260/max/82/2020/06/01...,
1,https://www.newsbomb.gr/kosmos/story/1122674/e...,Εκλογές ΗΠΑ: Έρχονται μέτρα στα επόμενα ντιμπέ...,2020-10-01,01:53:25+03:00,ΚΟΣΜΟΣ,Newsbomb,"Η επιτροπή, που είναι επιφορτισμένη με την οργ...",«Το πρώτο ντιμπέιτ κατέστησε σαφές το γεγονός ...,https://img.bbmd.gr/img/1260/max/82/2020/09/30...,AP Photo
2,https://www.newsbomb.gr/kosmos/story/1123295/h...,ΗΠΑ: Αρνητικός ο Τζο Μπάιντεν στο τεστ κορονοϊού,2020-10-02,20:05:24+03:00,ΚΟΣΜΟΣ,Newsbomb,Κορονοϊός - Όλες οι ειδήσεις: Ο Τζο Μπάιντεν κ...,Αρνητική στην Covid-19 βγήκε και η υποψήφια αν...,https://img.bbmd.gr/img/1260/max/82/2020/10/02...,
3,https://www.newsbomb.gr/kosmos/story/1123251/o...,ΗΠΑ: Ο Τζο Μπάιντεν εύχεται «ταχεία ανάρρωση» ...,2020-10-02,16:48:48+03:00,ΚΟΣΜΟΣ,Newsbomb,Κορονοϊός - Όλες οι ειδήσεις: Ο υποψήφιος των ...,Η Τζιλ Μπάιντεν (η σύζυγός του) και εγώ στέλνο...,https://img.bbmd.gr/img/1260/max/82/2020/10/02...,
4,https://www.newsbomb.gr/kosmos/story/1123371/h...,ΗΠΑ: «Ο Θεός να σε ευλογεί» - Το μήνυμα του Μπ...,2020-10-03,03:20:10+03:00,ΚΟΣΜΟΣ,Newsbomb,Ο Τζο Μπάιντεν έστειλε το δικό του μήνυμα στον...,«Αυτές είναι οι Ηνωμένες Πολιτείες της Αμερική...,https://img.bbmd.gr/img/1260/max/82/2020/10/03...,
5,https://www.newsbomb.gr/kosmos/story/1132775/e...,Εκλογές ΗΠΑ 2020 - Δημοσκόπηση: Προβάδισμα 7 μ...,2020-11-01,21:43:04+02:00,ΚΟΣΜΟΣ,Newsbomb,Εκλογές ΗΠΑ 2020: Στην τελική ευθεία η μάχη Τρ...,Με τις αμερικανικές προεδρικές εκλογές να απέχ...,https://img.bbmd.gr/img/1260/max/82/2020/11/01...,AP
6,https://www.newsbomb.gr/kosmos/story/1132686/e...,Εκλογές ΗΠΑ 2020: Τραμπ εναντίον Μπάιντεν – Τι...,2020-11-01,14:25:49+02:00,ΚΟΣΜΟΣ,Χρήστος Γιαννούλης,Προεδρικές Εκλογές ΗΠΑ 2020: Ένα 48ωρο πριν απ...,Οι ψηφοφόροι στις ΗΠΑ θα αποφασίσουν στις 3 Νο...,https://img.bbmd.gr/img/1260/max/82/2020/11/01...,Τι δείχνουν οι δημοσκοπήσεις για τις εκλογές τ...
7,https://www.newsbomb.gr/kosmos/story/1132681/e...,Εκλογές ΗΠΑ 2020: Ο Τραμπ επιτίθεται στον «κοι...,2020-11-01,13:55:24+02:00,ΚΟΣΜΟΣ,Χρήστος Γιαννούλης,Εκλογές ΗΠΑ 2020: Ο Ντόναλντ Τραμπ υποστήριξε ...,Η μάχη τωνΠροεδρικών Εκλογών στις ΗΠΑκορυφώνετ...,https://img.bbmd.gr/img/1260/max/82/2020/11/01...,"Ο Αμερικανός πρόεδρος, Ντόναλντ Τραμπ\nAP Images"
8,https://www.newsbomb.gr/kosmos/story/1132645/e...,Εκλογές ΗΠΑ 2020: Κορυφώνεται η μάχη Τραμπ vs ...,2020-11-01,11:16:56+02:00,ΚΟΣΜΟΣ,Χρήστος Γιαννούλης,Εκλογές ΗΠΑ 2020: Δυο ημέρες πριν κλείσουν οι ...,«Φωτιά» έχει πάρει το επιτελείο τουΝτόναλντ Τρ...,https://img.bbmd.gr/img/1260/max/82/2020/11/01...,Λιγότερες από 48 ώρες για την τελική αναμέτρησ...
9,https://www.newsbomb.gr/kosmos/story/1132276/e...,Εκλογές ΗΠΑ 2020: Τραμπ vs Μπάιντεν - Ποιος θα...,2020-11-01,08:27:57+02:00,ΚΟΣΜΟΣ,Ελένη Κολιοφούτη,Η αντίστροφη μέτρηση για τις αμερικανικές προε...,Αντίστροφα μετράει η κλεψύδρα για την πολυαναμ...,https://img.bbmd.gr/img/1260/max/82/2020/10/30...,Στην τελική ευθεία η μάχη Τραμπ - Μπάιντεν για...


In [ ]:
from google.colab import drive

In [ ]:
scrape_newsbomb_articles_pages(biden_teasers_df).to_csv('/content/drive/MyDrive/biden_articles_df_2020.csv', index=False)

Processing rows: 100%|██████████| 46/46 [02:09<00:00,  2.81s/it]


Scraping completed. Fetshed 46 articles.
